In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os

os.getcwd().split("/")[-1]
pdf_path = (
    "/".join(os.getcwd().split("/")[:-1]) + "/" + "data/Bank_eStmt_2025-07-14.pdf"
)

In [3]:
from pydantic import BaseModel

from doc_intelligence.llm import OpenAILLM
from doc_intelligence.processer import DocumentProcessor

In [4]:
llm = OpenAILLM()

In [5]:
processor = DocumentProcessor.from_digital_pdf(uri=pdf_path, llm=llm)

In [6]:
class Balance(BaseModel):
    ending_balance: float

In [7]:
config = {
    "response_format": Balance,
    "llm_config": {
        "model": "gpt-5",
        "reasoning": {"effort": "minimal"},
    },
    "extraction_config": {
        "include_citations": True,
        "extraction_mode": "single_pass",
        "page_numbers": [0, 1],
    },
}

In [8]:
response = processor.extract(config)

2026-02-09 08:48:29.661 | INFO     | doc_intelligence.processer:parse:47 - Document parsed successfully
2026-02-09 08:48:29.661 | DEBUG    | doc_intelligence.extractor:extract:48 - DigitalPDFExtractor: extract: json_instance_schema: {
    "ending_balance": {
        "value": <number>,
        "citations": [{"page": <integer>, "lines": [<integer>]}]
    }
}
2026-02-09 08:48:29.661 | INFO     | doc_intelligence.formatter:format_document_for_llm:53 - Formatting 2 pages
2026-02-09 08:48:29.662 | DEBUG    | doc_intelligence.extractor:extract:54 - DigitalPDFExtractor: extract: content_text: <page number=0>
0: P.O. Box 15284 Customer service information
1: Wilmington, DE 19850
2: Customer service: 1.800.432.1000
3: En Español: 1.800.688.6086
4: ZEEL DIPENKUMAR THUMAR bankofamerica.com
5: 230 CAMPBELL AVE
6: WEST HAVEN, CT 06516-5338 Bank of America, N.A.
7: P.O. Box 25118
8: Tampa, FL 33622-5118
9: Your Adv SafeBalance Banking
10: for June 11, 2025 to July 14, 2025 Account number: 3850 3327 1

In [9]:
response

(Balance(ending_balance=111.61),
 {'ending_balance': {'value': 111.61,
   'citations': [{'page': 0,
     'bboxes': [{'x0': 0.058823529411764705,
       'top': 0.6095707475757575,
       'x1': 0.5635455037254902,
       'bottom': 0.6221969596969696}]}]}})